In [3]:
import ee, geetools

ee.Initialize()

image = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(ee.Geometry.Point([12.4534, 41.9033]).buffer(1000))
    .filterDate("2023-01-01", "2023-01-31")
    .first()
    .select("B4", "B3", "B2")
)
hasattr(image.geetools, "isletMask")

True

In [5]:
from tempfile import NamedTemporaryFile

with NamedTemporaryFile() as tmp:
    print(tmp.name)

/tmp/tmpocq6b3zj
